In [1]:
import pandas as pd

In [ ]:
categories = pd.read_csv('data/categories.csv')

In [4]:
categories.head()

,Unnamed: 0,category_id,category_name,parent_category,created_at
0,0,1,Smartphones,ELECTRONICS,NaN
1,1,2,Laptops,Electronics,2024-08-09T15:21:20.057392
2,2,3,Tablets,NaN,2024-08-25T14:12:07.136934
3,3,4,Cameras,NaN,2024-07-02T05:37:42.784417
4,4,5,Televisions,Electronics,2024-09-11T15:18:54.079818


In [ ]:
n_cat = categories['category_name'].count()
print(f'Cantidad de categorias: {n_cat}')

AttributeError: 'numpy.int64' object has no attribute 'print'